# Les réseaux de Convolution

Les réseaux de convolution ont été introduits dès la fin des années avec les travaux de Yann LeCun. 
Ces derniers sont destinés au traitement de l'information organisé sous une forme de tableau 2D. Cette topologie regroupe aussi bien les images que les séries temporelles.

Cette approche fait aujourd'hui l'unanimité pour les problèmes de traitements d'images. Son succès tient à l'ajustement des méthodes de Deep Learning aux spécificités des images.

Dans cette partie, nous rappellerons tout d'abord la définition mathématique de l'opération de convolution. Nous expliciterons ensuite l'analogie entre la convolution mathématique et la convolution des réseaux neuronaux. Nous présentenrons alors les spécificités et le formalisme de cette approche.

## L'opération de convolution

L'opération de convolution est une opération bien connue notamment dans les sciences du traitement du signal. C'est cet opérateur qui régit la transformation d'un signal par un opérateur linéaire. 

Mathématiquement, l'opération s'écrit comme suit :
$$ s(t) = \int x(a)\omega(t-a)da $$ 

Forme qu'on abrège le plus souvent par : 
$$ s(t) = (x*\omega)(t) $$

Dans l'ensemble de nos applications, nous travaillons dans le domaine discret, il convient donc d'adopter la notation :
    $$ s(t) = \sum_{a=-\infty}^{\infty}x(a)\omega(t-a)   $$

De plus, dans nous travaillerons sur des images, soit des données 2D. 
Il convient alors de préciser l'écriture précédente :

$$ S(i,j) = (I*K)(i,j) = \sum_{m} \sum_{n} I(m,n)K(i-m,j-n) $$ 


avec:
- $I$ l'entrée
- $K$ le kernel

Enfin, on peut remarquer que l'inversion réalisée par la convolution n'est utile que pour garantir la commutativité de l'opération. Or, en pratique, cette propriété n'est pas utile en Apprentissage Machine.
Par conséquent, un certain nombre de librairies ont plutôt implémenté la *cross-correlation* qui n'inverse pas le kernel. 

## Quelles motivations pour les réseaux de convolution ?

Le succès des réseaux de convolution s'explique par trois propriétés particulières.

### Sparse Interactions

Dans une couche d'un réseau de neurones traditionnels, toutes les unités de l'*input* sont prises en compte pour calculer l'*output*. C'est pourquoi on parle de connexions denses.

Dans les réseaux de convolution, la situation est différente. L'idée est ici d'utiliser des *kernels* plus petits que l'*input*. Ces derniers seront consacrés à des tâches bien particulières; ne nécessitant qu'une information locale. Ainsi, même si l'on s'intéresse à une image composée de millons de pixels, il pourrait être pertinent d'utiliser des kernels que de quelques dizaines ou centaines de pixels de bord pour détecter les bordures.

Cette modélisation s'appuie donc sur la nature locale des informations contenues dans une image. Pour reconnaitre une roue, il n'y a toujours besoin de reconnaitre d'abord la voiture.
Ce choix permet d'effectuer des gains significatifs sur la mémoire utilisée et sur le nombre de calcul réalisé.

![sparseInteractions](img/sparseConnectivity.png)

### Parameter sharing

La notion de partage de paramètres définit la manière dont les paramètres vont être appris. Dans un réseau classique, chaque paramètre est calculé exactement une seule fois : lorsque un neurone particulier considère une entrée particulière. 
Désormais, avec le *paramater sharing*, les paramètres sont partagés entre les différents neurones. 

Ainsi, dans une couche consacrée à la détection des bords, deux neurones successifs appliqueront les mêmes paramètres à leurs entrées respectives. Ils sont destinés à détecter des bords, il est donc naturel qu'ils opérent les mêmes opérations sur leurs entrées.

Ce partage permet une économie très importante des puissances de calcul.

![gifConvolution](https://s1.qwant.com/thumbr/0x0/a/c/ba1e79eb93de2dc3f2891b4b4f1dfc/b_1_q_0_p_0.jpg?u=http%3A%2F%2Fbamos.github.io%2Fdata%2F2016-08-09%2Fpadding_strides.gif&q=0&b=1&p=0&a=1)

L'un des avantages du parameter sharing est qu'il rend le réseau insensible aux translations dans l'espace.

## Pooling

L'indépendance aux translations est un aspect fondamental des réseaux de conovolution. Il permet de vraiment se concentrer sur la présence d'un item plutôt que sur sa localisation.

![pooling](img/pooling.png)

### Field of View


Il faut ici adopter une vision plus globale de notre réseau. En effet, le pooling et le stride amènent les couches supérieures à percevoir une zone plus grande de l'image. 

En effet, au niveau $n$, le pooling résume les informations contenues dans un certain nombre de valeurs, `4` si nous reprenons l'exemple précédent.
La couche $n+1$ convoluera les valeurs contenues dans un certain espace, par exemple un carré `2x2`. 


Par conséquent la couche $n+1$ travaillera sur le résumé des informations de la couche précédente. Son filtre est de taille `2x2` au niveau $n+1$, mais il représente une taille `4x4` au niveau $n$.

![receiptiveField](https://cdn-images-1.medium.com/max/2000/1*KFX5mWoRMfMme2jngak8wg.png)

### Interprétation des différentes fonctions

### Formalisme mathématiques 

### Exemple rapide d'implémentation

Nous allons ici présenter une implémentation d'une couche de convolution, en utilisant la librairie <a href=index.html >TensorFlow</a>. 

La déclaration d'une couche peut tenir en seulement quelques lignes, comme nous pouvons le voir ci-dessous : 

`
""" Layer 1 """
W_conv1_1 = tf.Variable(tf.truncated_normal([3, 3, 3, 64],stddev=0.02))
b_conv1_1 = tf.Variable(tf.constant(0.0,[64])
conv1_1 = tf.nn.conv2d(image, W_conv1_1, strides=[1, s, s, 1], padding='SAME') + b_conv1_1
relu1_1 = tf.nn.relu(conv1_1)
pool1 = tf.nn.max_pool(relu1_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
`

La 1ère ligne définit la forme de la couche. Nous définissons alors dans l'ordre :
- la taille de la fenetre de convolution : ici `3x3` 
- la largeur de l'entrée : ici pour `3`, pour chaque catégorie de couleur
- la largeur de la sortie : ici `64`.

Dans la 2ème ligne, nous définissons le nombre de biais, qui est égal à la profondeur de la sortie, soit `64`.

Dans la 3ème ligne, nous définissons l'opération de convolution, en précisant l'image à filtrer, le filtre et les biais à utiliser.

Dans la 4ème ligne, nous définissons la fonction d'activation, ici la fonction `relu` classique.

Dans la 5ème ligne, nous définissons la fonction `pool` à utiliser.

### Exemple d'architecture